<center><strong><font size=+3>Explicit calibration of H1C_IDR2.2 visibilities</font></center>
<br><br>
</center>
<center><strong><font size=+2>Matyas Molnar</font><br></strong></center>
<br><center><strong><font size=+1>Astrophysics Group, Cavendish Laboratory, University of Cambridge</font></strong></center>

Notebook to show how to explicitly calibrate HERA datasets in `uvh5` file format, using the calibration functions from the [hera_cal](https://github.com/HERA-Team/hera_cal) package.

In [ ]:
import glob
import os

import numpy as np
from matplotlib import pyplot as plt

from hera_cal.abscal import post_redcal_abscal_run
from hera_cal.apply_cal import apply_cal
from hera_cal.io import HERAData, HERACal
from hera_cal.redcal import get_reds, redcal_run

from cal_utils import plot_vis, waterfall_column

In [ ]:
%matplotlib inline

In [ ]:
JD = 2458098.43869

In [ ]:
analysis_folder = '/lustre/aoc/projects/hera/H1C_IDR2/IDR2_2/{}'.format(int(JD))
raw_data_fn = 'zen.{}.HH.uvh5'.format(JD)

hera_pkgs = '/users/heramgr/hera_software/'
if not os.path.exists(analysis_folder): # working locally
    analysis_folder = '/Users/matyasmolnar/Downloads/HERA_Data/hc_analysis/sample_calib'
    hera_pkgs = '/Users/matyasmolnar/Downloads/HERA_Data/hera_packages/'

## Raw visibilities

In [ ]:
raw_data_file = os.path.join(analysis_folder, raw_data_fn)
# this is the baseline and file we examine through this notebook
bl = (12, 13, 'ee')#(38, 39, 'ee')

In [ ]:
hd = HERAData(raw_data_file)
data, flags, nsamples = hd.read(bls=[bl]) # only loads a single bl; default loads all bls

In [ ]:
plot_vis(data, flags, hd, JD, bl, 'phase', title='Raw Visibility Phases')

In [ ]:
plot_vis(data, flags, hd, JD, bl, 'amp', title='Raw Visibility Amplitudes')

## Redundant calibration

In [ ]:
bad_ants_file = os.path.join(hera_pkgs, 'hera_pipelines/pipelines/h1c/idr2/v2/bad_ants/{}.txt'.format(int(JD)))

bad_ants = np.loadtxt(bad_ants_file).astype(int)
print('Bad antennas for JD {} are: {}'.format(int(JD), list(bad_ants)))

In [ ]:
omni_vis_fn = raw_data_fn.replace('uvh5', 'omni_vis.uvh5')
omni_vis_file = os.path.join(analysis_folder, omni_vis_fn)

if not os.path.exists(omni_vis_file):
    print('Running REDCAL on raw visibility file')
    ant_z_thresh = 4.0
    solar_horizon = 0.0
    nInt_to_load = 8
    flag_nchan_low = 50
    flag_nchan_high = 50
    min_bl_cut = 15
    max_bl_cut = 90
    ex_ants = bad_ants
    outdir = analysis_folder

    redcal_run(raw_data_file, filetype='uvh5', firstcal_ext='.first.calfits', omnical_ext='.omni.calfits',
            omnivis_ext='.omni_vis.uvh5', iter0_prefix='', outdir=outdir, ant_metrics_file=None, clobber=True, 
            nInt_to_load=nInt_to_load, pol_mode='2pol', bl_error_tol=1.0, ex_ants=ex_ants, ant_z_thresh=ant_z_thresh, 
            max_rerun=5, solar_horizon=solar_horizon, flag_nchan_low=flag_nchan_low, flag_nchan_high=flag_nchan_high, 
            fc_conv_crit=1e-6, fc_maxiter=50, oc_conv_crit=1e-10, oc_maxiter=500, check_every=10, check_after=50, gain=.4, 
            add_to_history='', verbose=True, min_bl_cut=min_bl_cut, max_bl_cut=max_bl_cut)
else:
    print('REDCAL omni calibrated file already exists')

In [ ]:
hd = HERAData(omni_vis_file)

In [ ]:
# recall that at this stage, after redcal, all baselines of the same type are aggregated into baseline groups
# with the baseline group nameed after the first baseline in that group type
reds = get_reds(hd.antpos, pols=['ee'])
red_dict = {bl: bl_group[0] for bl_group in reds for bl in bl_group}
bl_red = red_dict[bl]
print('Baseline {} corresponds to redundant baseline group of type {}'.format(bl, bl_red))

In [ ]:
# lists of available baselines
selected_bls = [bl for bl, red_bl in red_dict.items() if red_bl == bl_red]
selected_bls_flagged = [bl for bl in selected_bls if not any(j in bad_ants for j in bl[:2])]

In [ ]:
# load redundant baseline of type bl_red
redcal_data, redcal_flags, redcal_nsamples = hd.read(bls=bl_red)

In [ ]:
plot_vis(redcal_data, redcal_flags, hd, JD, bl_red, 'phase', \
         title='Omni Calibrated Visibility Phases of Redundant Group')

In [ ]:
plot_vis(redcal_data, redcal_flags, hd, JD, bl_red, 'amp', \
         title='Omni Calibrated Visibility Amplitudes of Redundant Group')

## Absolute calibration

In [ ]:
omni_cal_file = omni_vis_file.replace('omni_vis.uvh5', 'omni.calfits')
abs_vis_fn = raw_data_fn.replace('uvh5', 'abs_vis.uvh5')
abs_vis_file = os.path.join(analysis_folder, abs_vis_fn)
abs_cal_file = abs_vis_file.replace('abs_vis.uvh5', 'abs.calfits')

if not os.path.exists(abs_cal_file):
    print('Running ABSCAL on redundantly calibrated visibility file')
    
    # Find corresponding LSTs in the externally calibrated Abscal Model
    model_dir = '/lustre/aoc/projects/hera/nkern/idr3_abscal_models/full_model/' # at NRAO
    if not os.path.exists(model_dir):
        model_dir = analysis_folder # local copy
    # cheatingly slightly by only checking the .5* files
    model_files = glob.glob(os.path.join(model_dir, 'zen.2458042.*.HH.uvRXLS.uvh5'))
    
    nInt_to_load = 6
    min_bl_cut = 1.0
    max_bl_cut = 100.0
    phs_max_iter = 100
    phs_conv_crit = 1e-6
    edge_cut = 100

    # ABSCAL output file is a calfits file
    post_redcal_abscal_run(raw_data_file, omni_cal_file, model_files, raw_auto_file=None, data_is_redsol=False, \
                           model_is_redundant=False, output_file=abs_cal_file, nInt_to_load=nInt_to_load, data_solar_horizon=90, \
                           model_solar_horizon=90, extrap_limit=.5, min_bl_cut=min_bl_cut, max_bl_cut=max_bl_cut, edge_cut=edge_cut, \
                           tol=1.0, phs_max_iter=phs_max_iter, phs_conv_crit=phs_conv_crit, refant=None, clobber=True, add_to_history='', \
                           verbose=True)
else:
    print('ABSCAL calibrated file already exists')

In [ ]:
if not os.path.exists(abs_vis_file):
    apply_cal(raw_data_file, abs_vis_file, abs_cal_file, old_calibration=None)

In [ ]:
hd = HERAData(abs_vis_file)
abscal_data, abscal_flags, abscal_nsamples = hd.read(bls=[bl])

In [ ]:
plot_vis(abscal_data, abscal_flags, hd, JD, bl, 'phase', \
         title='Absolutely Calibrated Visibility Phases of Redundant Group')

In [ ]:
plot_vis(abscal_data, abscal_flags, hd, JD, bl, 'amp', \
         title='Absolutely Calibrated Visibility Amplitudes of Redundant Group', \
         vmax=100)